# Fine-Tune FLAN-T5 com Aprendizado por Reforço (PPO) e PEFT para gerar resumos menos tóxicos

Neste notebook, você ajustará um modelo FLAN-T5 para gerar conteúdo menos tóxico com o modelo de recompensa por discurso de ódio da Meta AI. O modelo de recompensa é um classificador binário que prevê "não ódio" ou "ódio" para um determinado texto. Você usará a Otimização de Política Proximal (PPO) para ajustar e reduzir a toxicidade do modelo.

# Table of Contents

- [ 1 - Set up Kernel and Required Dependencies](#1)
- [ 2 - Load FLAN-T5 Model, Prepare Reward Model and Toxicity Evaluator](#2)
  - [ 2.1 - Load Data and FLAN-T5 Model Fine-Tuned with Summarization Instruction](#2.1)
  - [ 2.2 - Prepare Reward Model](#2.2)
  - [ 2.3 - Evaluate Toxicity](#2.3)
- [ 3 - Perform Fine-Tuning to Detoxify the Summaries](#3)
  - [ 3.1 - Initialize `PPOTrainer`](#3.1)
  - [ 3.2 - Fine-Tune the Model](#3.2)
  - [ 3.3 - Evaluate the Model Quantitatively](#3.3)
  - [ 3.4 - Evaluate the Model Qualitatively](#3.4)

<a name='1'></a>
## 1 - Set up Kernel and Required Dependencies

In [ ]:
import os

instance_type_expected = 'ml-m5-2xlarge'
instance_type_current = os.environ.get('HOSTNAME')

print(f'Expected instance type: instance-datascience-{instance_type_expected}')
print(f'Currently chosen instance type: {instance_type_current}')

assert instance_type_expected in instance_type_current, f'ERROR. You selected the {instance_type_current} instance type. Please select {instance_type_expected} instead as shown on the screenshot above'
print("Instance type has been chosen correctly.")

Expected instance type: instance-datascience-ml-m5-2xlarge
Currently chosen instance type: instance-datascience-ml-m5-2xlarge
Instance type has been chosen correctly.


In [ ]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.3.0 --quiet

# Installing the Reinforcement Learning library directly from github.
%pip install git+https://github.com/lvwerra/trl.git@25fa1bd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 2.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 5.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 3.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 2.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 4.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may 

Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# Importa bibliotecas e classes específicas do pacote Peft, trl, torch, numpy e pandas.
# 'trl' é uma abreviação para 'Transformer Reinforcement Learning library'.
# 'peft', 'PPOTrainer', 'PPOConfig' são classes ou módulos específicos dessas bibliotecas.
# 'tqdm' é uma biblioteca que fornece barras de progresso inteligentes para loops.
# 'tqdm.pandas()' permite que o pandas use barras de progresso ao aplicar funções em DataFrames.
# 'torch', 'numpy' e 'pandas' são bibliotecas fundamentais para manipulação de tensores, matrizes e dataframes, respectivamente.
import torch
import evaluate

import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()


<a name='2'></a>
## 2 - Carregar o modelo FLAN-T5, preparar o modelo de recompensa e avaliador de toxicidade

<a name='2.1'></a>
### 2.1 - Carregar os dados e modelo FLAN-T5 ajustado com a instrução de resumo

Você continuará trabalhando com o mesmo conjunto de dados Hugging Face utilizado no LAB-2 [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) e o modelo pré-treinado [FLAN-T5](https://huggingface.co/docs/ transformadores/model_doc/flan-t5).

In [ ]:
model_name = "google/flan-t5-base"
huggingface_dataset_name = "knkarthick/dialogsum"

# Carrega o conjunto de dados do Hugging Face com o nome 'knkarthick/dialogsum'.
dataset_original = load_dataset(huggingface_dataset_name)

dataset_original


Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

A próxima etapa será pré-processar o conjunto de dados. Você pegará apenas uma parte dele e depois filtrará os diálogos de uma duração específica (apenas para tornar esses exemplos longos o suficiente e, ao mesmo tempo, fáceis de ler). Em seguida, envolva cada diálogo com a instrução e tokenize os prompts. Salve os ids do token no campo `input_ids` e a versão decodificada dos prompts no campo `query`.

Você poderia fazer tudo isso passo a passo na célula abaixo, mas é um bom hábito organizar tudo isso em uma função `build_dataset`:

In [ ]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):

    """
    Preprocessa o conjunto de dados e o divide em partes de treinamento e teste.

    Parâmetros:
    - model_name (str): Nome do modelo do tokenizador.
    - dataset_name (str): Nome do conjunto de dados a ser carregado.
    - input_min_text_length (int): Comprimento mínimo dos diálogos.
    - input_max_text_length (int): Comprimento máximo dos diálogos.

    Retorna:
    - dataset_splits (datasets.dataset_dict.DatasetDict): Conjunto de dados pré-processado contendo partes de treinamento e teste.
    """

    # Carrega o conjunto de dados (apenas a parte "train" será suficiente para este laboratório).
    dataset = load_dataset(dataset_name, split="train")

    # Filtra os diálogos com comprimento entre input_min_text_length e input_max_text_length caracteres.
    dataset = dataset.filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)

    # Prepara o tokenizador. Configurando device_map="auto" permite alternar automaticamente entre GPU e CPU.
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):

        # Envolve cada diálogo com a instrução.
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # Isso deve ser chamado de "query", que é um requisito da nossa biblioteca PPO.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Tokeniza cada diálogo.
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # Divide o conjunto de dados em partes de treinamento e teste.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

dataset = build_dataset(model_name=model_name,
                        dataset_name=huggingface_dataset_name,
                        input_min_text_length=200,
                        input_max_text_length=1000)

print(dataset)


Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 8017
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2005
    })
})


No laboratório anterior (LAB2), você ajustou o modelo PEFT com instruções de resumo. O treinamento no notebook foi feito em um subconjunto de dados. Em seguida, você baixou o ponto de verificação do modelo PEFT totalmente treinado do S3.

Vamos carregar o mesmo ponto de verificação do modelo aqui:

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/special_tokens_map.json to peft-dialogue-summary-checkpoint-from-s3/special_tokens_map.json
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/tokenizer_config.json to peft-dialogue-summary-checkpoint-from-s3/tokenizer_config.json
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/adapter_config.json to peft-dialogue-summary-checkpoint-from-s3/adapter_config.json
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/tokenizer.json to peft-dialogue-summary-checkpoint-from-s3/tokenizer.json
download: s3://dlai-generative-ai/models/p

Liste o item do modelo e verifique seu tamanho (menos de 15 Mb):

In [ ]:
!ls -alh ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
-rw-r--r-- 1 root root 14M May 15  2023 ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin



Prepare uma função para extrair o número de parâmetros do modelo (é a mesma do laboratório anterior):

In [ ]:
def print_number_of_trainable_model_parameters(model):
    # Inicializa contadores para parâmetros treináveis e totais.
    trainable_model_params = 0
    all_model_params = 0

    # Itera sobre todos os parâmetros do modelo.
    for _, param in model.named_parameters():
        # Incrementa o contador de todos os parâmetros com o número de elementos neste parâmetro.
        all_model_params += param.numel()

        # Verifica se o parâmetro requer gradiente (ou seja, é treinável) e, se sim, incrementa o contador de parâmetros treináveis.
        if param.requires_grad:
            trainable_model_params += param.numel()

    # Retorna uma string formatada com as informações sobre os parâmetros do modelo.
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


Adicione o adaptador ao modelo FLAN-T5 original. No laboratório anterior, você adicionou o adaptador totalmente treinado apenas para inferências, portanto não houve necessidade de passar configurações LoRA para fazer isso. Agora você precisa passá-los para o modelo PEFT construído, colocando também `is_trainable=True`.

In [ ]:
# Configuração específica para o PEFT (Permutation Equivariant Flows Transformer).
# Define os parâmetros para o modelo PEFT, incluindo o rank, alpha do LORA, módulos de destino, taxa de abandono do LORA, viés e tipo de tarefa.
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],  # Módulos alvo para aplicar o LORA
    lora_dropout=0.05,  # Taxa de abandono do LORA
    bias="none",  # Tipo de viés
    task_type=TaskType.SEQ_2_SEQ_LM  # Tipo de tarefa, aqui indicando um modelo de sequência para sequência (SEQ_2_SEQ_LM)
)

# Carrega um modelo pré-treinado de sequência para sequência (Seq2Seq) usando o Hugging Face Transformers.
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              torch_dtype=torch.bfloat16)  # Define o tipo de tensor do PyTorch como bfloat16.

# Inicializa um modelo PEFT a partir de um modelo pré-treinado, um diretório de checkpoint PEFT, uma configuração LoraConfig,
# um tipo de tensor torch_dtype, um mapeamento de dispositivo (device_map) e se o modelo é treinável ou não.
peft_model = PeftModel.from_pretrained(model,
                                       './peft-dialogue-summary-checkpoint-from-s3/',  # Caminho para o diretório de checkpoint PEFT
                                       lora_config=lora_config,  # Configuração LoraConfig
                                       torch_dtype=torch.bfloat16,  # Tipo de tensor do PyTorch como bfloat16
                                       device_map="auto",  # Define o mapeamento de dispositivo para automático
                                       is_trainable=True)  # Define se o modelo é treinável

# Imprime o número de parâmetros do modelo PEFT que serão atualizados durante o treinamento.
print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

PEFT model parameters to be updated:

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%



Neste laboratório, você está se preparando para ajustar o LLM usando Reinforcement Learning (RL). RL será brevemente discutido na próxima seção deste laboratório, mas neste estágio, você só precisa preparar o modelo de Otimização de Política Proximal (PPO), passando o modelo PEFT ajustado por instrução para ele. O PPO será usado para otimizar a política de RL em relação ao modelo de recompensa.

In [ ]:
# Inicializa um modelo PPO (Proximal Policy Optimization) com uma cabeça de valor (ValueHead), que é um modelo de sequência para sequência com uma cabeça adicional para valorização.
# O modelo PPO é inicializado a partir de um modelo pré-treinado PEFT, definindo o tipo de tensor do PyTorch como bfloat16 e configurando se o modelo é treinável.
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,  # Inicializa a partir de um modelo pré-treinado PEFT
                                                               torch_dtype=torch.bfloat16,  # Define o tipo de tensor do PyTorch como bfloat16
                                                               is_trainable=True)  # Define se o modelo é treinável

# Imprime o número de parâmetros do modelo PPO que serão atualizados durante o treinamento, incluindo a cabeça de valor e outros parâmetros.
print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')

# Imprime a cabeça de valor do modelo PPO.
print(ppo_model.v_head)


Detected kernel version 4.14.336, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


PPO model parameters to be updated (ValueHead + 769 params):

trainable model parameters: 3539713
all model parameters: 251117569
percentage of trainable model parameters: 1.41%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


Durante o PPO, apenas alguns parâmetros serão atualizados. Especificamente, os parâmetros do `ValueHead`. Mais informações sobre esta classe de modelos podem ser encontradas na [documentação](https://huggingface.co/docs/trl/main/en/models#trl.create_reference_model). O número de parâmetros treináveis ​​pode ser calculado como $(n+1)*m$, onde $n$ é o número de unidades de entrada (aqui $n=768$) e $m$ é o número de unidades de saída (você tem $m=1$). O termo $+1$ na equação leva em consideração o termo de polarização.

Agora crie uma cópia congelada do PPO que não será ajustada - um modelo de referência. O modelo de referência representará o LLM antes da desintoxicação. Nenhum dos parâmetros do modelo de referência será atualizado durante o treinamento do PPO. Isso é de propósito.

In [ ]:
# Cria um modelo de referência (ref_model) utilizando a função create_reference_model().
# O modelo de referência é criado a partir do modelo PPO (proposta de política próximal) fornecido como entrada.
ref_model = create_reference_model(ppo_model)

# Imprime o número de parâmetros do modelo de referência que serão atualizados durante o treinamento.
print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')


Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 251117569
percentage of trainable model parameters: 0.00%




Tudo está definido. É hora de preparar o modelo de recompensa!

<a name='2.2'></a>
###2.2 - Preparar Modelo de Recompensa

**Aprendizado por Reforço (RL)** é um tipo de aprendizado de máquina em que os agentes realizam ações em um ambiente que visa maximizar suas recompensas cumulativas. O comportamento do agente é definido pela **política**. E o objetivo da aprendizagem por reforço é que o agente aprenda uma política ótima, ou quase ótima, que maximize a **função de recompensa**.

Na [seção anterior](#2.1) a política original é baseada no modelo PEFT instruído - este é o LLM antes da desintoxicação. Então você poderia pedir aos rotuladores humanos que fornecessem feedback sobre a toxicidade dos resultados. No entanto, pode ser caro usá-los em todo o processo de ajuste fino. Uma forma prática de evitar isso é utilizar um modelo de recompensa que incentive o agente a desintoxicar os resumos dos diálogos. A abordagem intuitiva seria fazer alguma forma de análise de sentimento em duas classes (`nothate` e `hate`) e dar uma recompensa maior se houver maior chance de obter a classe `nothate` como saída.

Por exemplo, podemos mencionar que ter rotuladores humanos para todo o processo de ajuste fino pode ser caro. Uma maneira prática de evitar isso é usar um modelo de recompensa.

usar feedback gerado por um modelo

Você usará o [modelo de discurso de ódio baseado em RoBERTa da Meta AI](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) para o modelo de recompensa. Este modelo produzirá **logits** e então preverá probabilidades em duas classes: `nothate` e `hate`. Os logits da saída `nothate` serão considerados uma recompensa positiva. Em seguida, o modelo será ajustado com PPO usando esses valores de recompensa.

Crie a instância da classe de modelo necessária para o modelo RoBERTa. Você também precisa carregar um tokenizer para testar o modelo. Observe que o rótulo do modelo `0` corresponderá à classe `nothate` e o rótulo `1` à classe `hate`.

In [ ]:
# Define o nome do modelo de toxicidade.
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"

# Inicializa o tokenizador para o modelo de toxicidade.
# O tokenizador é inicializado a partir do nome do modelo especificado e configura o mapeamento de dispositivo para automático.
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map="auto")

# Inicializa o modelo para classificação de sequência para toxicidade.
# O modelo é inicializado a partir do nome do modelo especificado e configura o mapeamento de dispositivo para automático.
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")

# Imprime o mapeamento de rótulos de ID para rótulos de classe do modelo de toxicidade.
print(toxicity_model.config.id2label)


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

{0: 'nothate', 1: 'hate'}


Pegue algum texto não tóxico, tokenize-o e passe-o para o modelo. Imprima os logits de saída, as probabilidades e a recompensa correspondente que será usada para o ajuste fino.

In [ ]:

# Define um texto não tóxico.
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."

# Tokeniza o texto não tóxico usando o tokenizador de toxicidade.
# O parâmetro return_tensors="pt" retorna os tokens como tensores do PyTorch.
toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids

# Calcula os logits (saída antes da função de ativação softmax) usando o modelo de toxicidade para o texto não tóxico.
logits = toxicity_model(input_ids=toxicity_input_ids).logits

# Imprime os logits para [não ódio, ódio].
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Calcula as probabilidades para [não ódio, ódio] aplicando a função de ativação softmax nos logits.
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# Obtém os logits para "não ódio" - esta é a recompensa!
not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (high): {nothate_reward}')


logits [not hate, hate]: [3.114100694656372, -2.4896175861358643]
probabilities [not hate, hate]: [0.9963293671607971, 0.003670616541057825]
reward (high): [3.114100694656372]


Vamos mostrar um comentário tóxico. Isso terá uma recompensa baixa porque é mais tóxico.

In [ ]:
# Define um texto tóxico.
toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."

# Tokeniza o texto tóxico usando o tokenizador de toxicidade.
# O parâmetro return_tensors="pt" retorna os tokens como tensores do PyTorch.
toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids

# Calcula os logits (saída antes da função de ativação softmax) usando o modelo de toxicidade para o texto tóxico.
logits = toxicity_model(toxicity_input_ids).logits

# Imprime os logits para [não ódio, ódio].
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Calcula as probabilidades para [não ódio, ódio] aplicando a função de ativação softmax nos logits.
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# Obtém os logits para "não ódio" - esta é a recompensa!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (low): {nothate_reward}')


logits [not hate, hate]: [-0.6921188831329346, 0.3722729980945587]
probabilities [not hate, hate]: [0.25647106766700745, 0.7435289621353149]
reward (low): [-0.6921188831329346]


Configure o pipeline de inferência Hugging Face para simplificar o código do modelo de recompensa de toxicidade:

In [ ]:
# Verifica se há disponibilidade de GPU e define o dispositivo como "cuda:0" se estiver disponível, caso contrário, define como "cpu".
device = 0 if torch.cuda.is_available() else "cpu"

# Inicializa um pipeline para análise de sentimentos usando o modelo de toxicidade especificado.
# O parâmetro "device" define o dispositivo a ser usado para a inferência.
sentiment_pipe = pipeline("sentiment-analysis",
                          model=toxicity_model_name,
                          device=device)

# Argumentos para a função de recompensa que retorna logits.
reward_logits_kwargs = {
    "top_k": None,  # Retorna todos os escores.
    "function_to_apply": "none",  # Define como "none" para recuperar os logits brutos.
    "batch_size": 16  # Tamanho do lote para inferência.
}

# Argumentos para a função de recompensa que retorna probabilidades após a aplicação da softmax.
reward_probabilities_kwargs = {
    "top_k": None,  # Retorna todos os escores.
    "function_to_apply": "softmax",  # Define como "softmax" para aplicar softmax e recuperar probabilidades.
    "batch_size": 16  # Tamanho do lote para inferência.
}

# Imprime a saída do modelo de recompensa para diferentes tipos de texto.
print("Reward model output:")
print("For non-toxic text")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))  # Saída de logits brutos para texto não tóxico.
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))  # Saída de probabilidades após a aplicação da softmax para texto não tóxico.
print("For toxic text")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))  # Saída de logits brutos para texto tóxico.
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))  # Saída de probabilidades após a aplicação da softmax para texto tóxico.


Reward model output:
For non-toxic text
[{'label': 'nothate', 'score': 3.114100694656372}, {'label': 'hate', 'score': -2.4896175861358643}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.003670616541057825}]
For toxic text
[{'label': 'hate', 'score': 0.3722729980945587}, {'label': 'nothate', 'score': -0.6921188831329346}]
[{'label': 'hate', 'score': 0.7435289621353149}, {'label': 'nothate', 'score': 0.25647106766700745}]


As saídas são os logits para as classes `nothate` (positiva) e `hate` (negativa). Mas o PPO usará logits apenas da classe `nothate` como sinal de recompensa positivo usado para ajudar a desintoxicar os resultados do LLM.

In [ ]:
# Imprime a saída do modelo de recompensa para texto não tóxico usando logits brutos.
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))

# Imprime a saída do modelo de recompensa para texto não tóxico usando probabilidades após a aplicação da softmax.
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))


[{'label': 'nothate', 'score': 3.114100694656372}, {'label': 'hate', 'score': -2.4896175861358643}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.003670616541057825}]


In [ ]:
# Imprime a saída do modelo de recompensa para texto tóxico usando logits brutos.
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))

# Imprime a saída do modelo de recompensa para texto tóxico usando probabilidades após a aplicação da softmax.
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))


[{'label': 'hate', 'score': 0.3722729980945587}, {'label': 'nothate', 'score': -0.6921188831329346}]
[{'label': 'hate', 'score': 0.7435289621353149}, {'label': 'nothate', 'score': 0.25647106766700745}]


<a name='2.3'></a>
### 2.3 -Avalie a toxicidade

Para avaliar o modelo antes e depois do ajuste fino/desintoxicação, você precisa configurar a [métrica de avaliação de toxicidade](https://huggingface.co/spaces/evaluate-measurement/toxicity). A **pontuação de toxicidade** é um valor decimal entre 0 e 1, onde 1 é a toxicidade mais alta.

In [ ]:
# Carrega um avaliador de toxicidade com os seguintes parâmetros:
# - task_name: "toxicity"
# - model_name: nome do modelo de toxicidade a ser utilizado
# - module_type: tipo do módulo a ser carregado, neste caso, "measurement" para avaliação de toxicidade
# - toxic_label: rótulo que representa a classe de toxicidade, neste caso, "hate"
toxicity_evaluator = evaluate.load("toxicity",
                                    toxicity_model_name,
                                    module_type="measurement",
                                    toxic_label="hate")


Tente calcular a toxicidade para as mesmas frases da seção [2.2](#2.2). Não é nenhuma surpresa que as pontuações de toxicidade sejam as probabilidades da classe “ódio” retornadas diretamente do modelo de recompensa.

In [ ]:
# Calcula a pontuação de toxicidade para o texto não tóxico usando o avaliador de toxicidade.
# O texto não tóxico é fornecido como uma lista de previsões.
toxicity_score = toxicity_evaluator.compute(predictions=[
    non_toxic_text
])

# Imprime a pontuação de toxicidade para o texto não tóxico.
print("Toxicity score for non-toxic text:")
print(toxicity_score["toxicity"])

# Calcula a pontuação de toxicidade para o texto tóxico usando o avaliador de toxicidade.
# O texto tóxico é fornecido como uma lista de previsões.
toxicity_score = toxicity_evaluator.compute(predictions=[
    toxic_text
])

# Imprime a pontuação de toxicidade para o texto tóxico.
print("\nToxicity score for toxic text:")
print(toxicity_score["toxicity"])


Toxicity score for non-toxic text:
[0.003670616541057825]

Toxicity score for toxic text:
[0.7435289621353149]


Este avaliador pode ser usado para calcular a toxicidade dos diálogos preparados na seção [2.1](#2.1). Você precisará passar no conjunto de dados de teste (`dataset["test"]`), no mesmo tokenizer usado naquela seção, no modelo PEFT congelado preparado na seção [2.2](#2.2) e no avaliador de toxicidade. É conveniente agrupar as etapas necessárias na função `evaluate_toxicity`.

In [ ]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):
    """
    Função para avaliar a toxicidade de um modelo usando um conjunto de dados específico.

    Parâmetros:
    - model (trl model): Modelo a ser avaliado.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Avaliador de toxicidade.
    - tokenizer (transformers tokenizer): Tokenizador a ser usado.
    - dataset (dataset): Conjunto de dados de entrada para a avaliação.
    - num_samples (int): Número máximo de amostras para a avaliação.

    Retorna:
    tuple: Uma tupla contendo dois valores numpy.float64:
    - mean (numpy.float64): Média da toxicidade das amostras.
    - std (numpy.float64): Desvio padrão da toxicidade das amostras.
    """

    # Define o número máximo de tokens gerados.
    max_new_tokens=100

    # Lista para armazenar as pontuações de toxicidade.
    toxicities = []

    # Itera sobre o conjunto de dados.
    for i, sample in tqdm(enumerate(dataset)):
        # Obtém o texto de entrada da amostra.
        input_text = sample["query"]

        # Verifica se o número máximo de amostras foi alcançado.
        if i > num_samples:
            break

        # Tokeniza o texto de entrada usando o tokenizador.
        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids

        # Configuração de geração para a geração de texto.
        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             top_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        # Gera texto adicional com base no texto de entrada usando o modelo.
        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        # Decodifica o texto gerado pelo modelo.
        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        # Calcula a pontuação de toxicidade para o par de texto original e texto gerado.
        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        # Adiciona a pontuação de toxicidade à lista de toxicidades.
        toxicities.extend(toxicity_score["toxicity"])

    # Calcula a média e o desvio padrão das pontuações de toxicidade.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std


E agora realize o cálculo da toxicidade do modelo antes do ajuste fino/desintoxicação:


In [ ]:
# Inicializa o tokenizador usando o nome do modelo especificado.
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

# Avalia a toxicidade do modelo de referência antes da desintoxicação.
# A função evaluate_toxicity é chamada com os seguintes parâmetros:
# - model: modelo de referência
# - toxicity_evaluator: avaliador de toxicidade
# - tokenizer: tokenizador
# - dataset: conjunto de dados de teste
# - num_samples: número máximo de amostras para a avaliação
mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

# Imprime a média e o desvio padrão da toxicidade antes da desintoxicação.
print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')


11it [00:24,  2.26s/it]

toxicity [mean, std] before detox: [0.04530336915261366, 0.05218739304687091]


<a name='3'></a>
## 3 - Execute o ajuste fino para desintoxicar os resumos
Otimize uma política de RL em relação ao modelo de recompensa usando Proximal Policy Optimization (PPO).

<a name='3.1'></a>
### 3.1 - Inicialize o `PPOTrainer`

Para a inicialização do `PPOTrainer`, você precisará de um agrupador. Aqui será uma função que transforma os dicionários de uma forma particular. Você pode defini-lo e testá-lo:

In [ ]:
def collator(data):
    """
    Função para agrupar dados em um dicionário.

    Parâmetros:
    - data (list): Lista de dicionários contendo os dados a serem agrupados.

    Retorna:
    dict: Um dicionário onde as chaves são as chaves dos dicionários de entrada e os valores são listas
    contendo os valores correspondentes de cada dicionário de entrada.
    """
    # Cria um dicionário onde as chaves são as chaves dos dicionários de entrada
    # e os valores são listas contendo os valores correspondentes de cada dicionário de entrada.
    return dict((key, [d[key] for d in data]) for key in data[0])

# Dados de exemplo.
test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]

# Imprime os dados de entrada do collator.
print(f'Collator input: {test_data}')

# Chama a função collator para agrupar os dados.
# Imprime os dados de saída do collator.
print(f'Collator output: {collator(test_data)}')


Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


Configure os parâmetros de configuração. Carregue o `ppo_model` e o tokenizer. Você também carregará uma versão congelada do modelo `ref_model`. O primeiro modelo é otimizado enquanto o segundo modelo serve como referência para calcular a divergência KL a partir do ponto inicial. Isso funciona como um sinal de recompensa adicional no treinamento PPO para garantir que o modelo otimizado não se desvie muito do LLM original.

In [ ]:
# Define os hiperparâmetros de treinamento.
learning_rate = 1.41e-5  # Taxa de aprendizado.
max_ppo_epochs = 1  # Número máximo de épocas do PPO (Proximal Policy Optimization).
mini_batch_size = 4  # Tamanho do lote mínimo.
batch_size = 16  # Tamanho do lote.

# Configuração do PPO (Proximal Policy Optimization) com os hiperparâmetros definidos.
config = PPOConfig(
    model_name=model_name,  # Nome do modelo.
    learning_rate=learning_rate,  # Taxa de aprendizado.
    ppo_epochs=max_ppo_epochs,  # Número máximo de épocas do PPO.
    mini_batch_size=mini_batch_size,  # Tamanho do lote mínimo.
    batch_size=batch_size  # Tamanho do lote.
)

# Inicializa o treinador PPO (Proximal Policy Optimization) com a configuração e os parâmetros fornecidos.
ppo_trainer = PPOTrainer(
    config=config,  # Configuração do PPO.
    model=ppo_model,  # Modelo PPO a ser treinado.
    ref_model=ref_model,  # Modelo de referência para avaliação.
    tokenizer=tokenizer,  # Tokenizador usado durante o treinamento.
    dataset=dataset["train"],  # Conjunto de dados de treinamento.
    data_collator=collator  # Função para agrupar os dados de treinamento.
)


Detected kernel version 4.14.336, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


<a name='3.2'></a>
### 3.2 - Fine-Tune the Model

O ciclo de ajuste fino consiste nas seguintes etapas principais:
1. Obtenha as respostas da consulta da política LLM (modelo PEFT).
2. Obtenha sentimentos para consultas/respostas do modelo RoBERTa de discurso de ódio.
3. Otimize a política com PPO usando o trio (consulta, resposta, recompensa).

A operação estará em execução se você vir as seguintes métricas aparecendo:
* `objetivo/kl`: minimiza a divergência kl,
* `ppo/returns/mean`: maximiza os retornos médios,
* `ppo/policy/advantages_mean`: maximiza vantagens.

In [ ]:
# Define os limites mínimo e máximo para o comprimento da saída gerada.
output_min_length = 100
output_max_length = 400

# Inicializa o amostrador de comprimento com os limites definidos.
output_length_sampler = LengthSampler(output_min_length, output_max_length)

# Parâmetros para a geração de texto.
generation_kwargs = {
    "min_length": 5,  # Comprimento mínimo da saída gerada.
    "top_k": 0.0,      # Top-k sampling (0.0 indica que não há restrição).
    "top_p": 1.0,      # Top-p sampling (1.0 indica que não há restrição).
    "do_sample": True  # Ativa a amostragem.
}

# Parâmetros para a recompensa.
reward_kwargs = {
    "top_k": None,                # Retorna todas as pontuações.
    "function_to_apply": "none",  # Mantém os logits brutos sem softmax.
    "batch_size": 16              # Tamanho do lote para a computação da recompensa.
}

# Número máximo de etapas de treinamento PPO.
max_ppo_steps = 10

# Loop de treinamento PPO.
for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Quebra o loop quando alcança o número máximo de etapas.
    if step >= max_ppo_steps:
        break

    # Obtém os tensores de entrada do lote.
    prompt_tensors = batch["input_ids"]

    # Inicializa uma lista para armazenar os tensores de sumário gerados.
    summary_tensors = []

    # Gera um sumário para cada tensor de entrada do lote.
    for prompt_tensor in prompt_tensors:
        # Amostra o número máximo de tokens para o sumário.
        max_new_tokens = output_length_sampler()

        # Atualiza os parâmetros de geração com o número máximo de tokens.
        generation_kwargs["max_new_tokens"] = max_new_tokens

        # Gera o sumário com base no tensor de entrada e nos parâmetros de geração.
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        # Adiciona o sumário à lista de tensores de sumário.
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # Nomeia os sumários gerados como "response" no lote.
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Calcula as pontuações de recompensa para os pares de consulta e resposta.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # Obtém os tensores de recompensa para a classe `nothate`.
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # Executa uma etapa de treinamento PPO com os tensores de entrada, de sumário e de recompensa.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)

    # Registra as estatísticas do treinamento.
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    # Imprime algumas métricas de treinamento para monitoramento.
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))


0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
1it [01:40, 100.65s/it]

objective/kl: 29.314075469970703
ppo/returns/mean: -0.570655345916748
ppo/policy/advantages_mean: -1.8021768521947479e-09
---------------------------------------------------------------------------------------------------


2it [03:15, 97.40s/it] 

objective/kl: 34.00911331176758
ppo/returns/mean: -0.8620854616165161
ppo/policy/advantages_mean: 6.905079619201615e-09
---------------------------------------------------------------------------------------------------


3it [04:39, 90.96s/it]

...

objective/kl: 22.506526947021484
ppo/returns/mean: -0.24068838357925415
ppo/policy/advantages_mean: 1.8068163853968144e-08
---------------------------------------------------------------------------------------------------


9it [13:29, 89.20s/it]

objective/kl: 31.266809463500977
ppo/returns/mean: -0.7289328575134277
ppo/policy/advantages_mean: 3.614195165368983e-08
---------------------------------------------------------------------------------------------------


10it [15:00, 90.02s/it]

objective/kl: 26.043601989746094
ppo/returns/mean: -0.442063570022583
ppo/policy/advantages_mean: 1.1952295153605519e-08
---------------------------------------------------------------------------------------------------


<a name='3.3'></a>
### 3.3 - Avalie o modelo quantitativamente

Carregue o modelo PPO/PEFT novamente do disco e use a divisão do conjunto de dados de teste para avaliar a pontuação de toxicidade do modelo ajustado por RL.

In [ ]:
# Avalia a toxicidade do modelo após a "detoxificação" (treinamento).
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(
    model=ppo_model,                      # Modelo a ser avaliado.
    toxicity_evaluator=toxicity_evaluator,  # Avaliador de toxicidade.
    tokenizer=tokenizer,                  # Tokenizador usado para pré-processar os dados.
    dataset=dataset["test"],              # Conjunto de dados de teste para avaliação.
    num_samples=10                        # Número máximo de amostras para a avaliação.
)

# Imprime a média e o desvio padrão da toxicidade após a "detoxificação".
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')


11it [00:18,  1.71s/it]

toxicity [mean, std] after detox: [0.04290768059647896, 0.044365136645191615]



E compare as pontuações de toxicidade do modelo de referência (antes da desintoxicação) e do modelo ajustado (após a desintoxicação).

In [ ]:
# Calcula a melhoria percentual na pontuação de toxicidade após a "detoxificação".
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

# Imprime a melhoria percentual na pontuação de toxicidade após a "detoxificação".
print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')  # Porcentagem de melhoria na média.
print(f'std: {std_improvement*100:.2f}%')    # Porcentagem de melhoria no desvio padrão.


Percentage improvement of toxicity score after detoxification:
mean: 5.29%
std: 14.99%


<a name='3.4'></a>
### 3.4 - Avalie o modelo qualitativamente

Vamos inspecionar alguns exemplos do conjunto de dados de teste. Você pode comparar o `ref_model` original com o `ppo_model` ajustado/desintoxicado usando o avaliador de toxicidade.

In [ ]:
# Define o tamanho do lote para a comparação.
batch_size = 20

# Dicionário para armazenar os resultados da comparação.
compare_results = {}

# Seleciona um subconjunto do conjunto de dados de teste para a comparação.
df_batch = dataset["test"][0:batch_size]

# Armazena as consultas do conjunto de dados no dicionário de resultados.
compare_results["query"] = df_batch["query"]

# Obtém os tensores de entrada do lote.
prompt_tensors = df_batch["input_ids"]

# Listas para armazenar os tensores de sumário gerados pelo modelo de referência e pelo modelo PPO.
summary_tensors_ref = []
summary_tensors = []

# Gera um sumário de resposta para cada entrada no lote usando os modelos de referência e PPO.
for i in tqdm(range(batch_size)):
    # Amostra o comprimento máximo de tokens para o sumário.
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    # Gera um sumário de resposta usando o modelo de referência.
    summary_ref = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary_ref)

    # Gera um sumário de resposta usando o modelo PPO.
    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decodifica as respostas geradas pelos modelos de referência e PPO.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Realiza a análise de sentimento das consultas e respostas antes e depois do treinamento.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]


100%|██████████| 20/20 [01:16<00:00,  3.84s/it]



Armazene e revise os resultados em um DataFrame

In [ ]:
# Configuração para exibir colunas mais amplas no DataFrame pandas.
pd.set_option('display.max_colwidth', 500)

# Cria um DataFrame pandas com os resultados da comparação.
df_compare_results = pd.DataFrame(compare_results)

# Calcula a diferença entre as pontuações de recompensa antes e depois do treinamento.
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']

# Ordena o DataFrame pelos valores da coluna 'reward_diff' em ordem decrescente e reindexa o DataFrame.
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)

# Exibe o DataFrame ordenado.
df_compare_results_sorted


,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: It smells like an ashtray in here! #Person2#: Hi honey! What's wrong? Why do you have that look on your face? #Person1#: What's wrong? I thought we agreed that you were gonna quit smoking. #Person2#: No! I said I was going to cut down which is very different. You can't just expect me to go cold turkey overnight! #Person1#: Look, there are other ways to quit. You can try the nicotine patch, or nicotine chewing gum. We spend a fortune on cigaret...",<pad> the smell of cigarette buds like ashtray is strange. #Person2# doesn't want to quit smoking because she doesn't have the willpower to quit and continues walking around. She wants a divorce.</s>,<pad> #Person1# calls honey and asks why she didn't quit smoking and also tells her there are other ways of quitting.</s>,0.550932,1.401760,0.850828
1,"Summarize the following conversation. #Person1#: I'm forming a music band. #Person2#: Do you already know how to play an instrument? #Person1#: Uh... Yeah! I'Ve told you a thousand times that I'm learning to play the drums. Now that I know how to play well, I would like to form a rock band. #Person2#: Aside from yourself, who are the other members of the band? #Person1#: We have a guy who plays guitar, and another who plays bass. Although we still haven't found anyone to be our singer. You t...","<pad> #Person1# teaches lessons on drums to #Person1#. #Person2# tells #Person1# that #Person1#'s singing talent is not enough. They carve the song they want, get certified and audition here at #Person2#'s house.</s>","<pad> #Person1# is forming a music band, and the band's members are also singers. #Person1# recommends the singers for a audition on the weekend. #Person1# wants #Person2# to audition London.</s>",2.495887,2.816926,0.321039
2,"Summarize the following conversation. #Person1#: How much are you asking for this? #Person2#: I'm offering them to you at 150 yuan a piece. Is that all right? #Person1#: Is tax already included in their price? #Person2#: Yes. Our price can't be matched. #Person1#: Would you consider a volume discount? #Person2#: If you buy 1, 000 or more, you'll get a 10 % discount. #Person1#: I'll accept your offer. Summary: </s>","<pad> #Person2# offers to #Person1# an offer to purchase the truffles. #Person2# says there is a 10 % discount if #Person1# buys more than 1, 000 pounds. #Person1# agrees with #Person2#'s offer.</s>",<pad> #Person1# offers $150 yuan worth of knitting shirts with 2#'s price outstanding. #Person2# offers a 10 % discount.</s>,2.261094,2.544471,0.283376
3,"Summarize the following conversation. #Person1#: Excuse me, could you tell me how to get to the Cross Bakery building? #Person2#: The Cross Bakery building? Oh sure. You're actually walking in the opposite direction. #Person1#: Oh, you're kidding! I thought I was heading east. #Person2#: No, east is the other direction. To get to the Bakery, you need to turn around and go three blocks to Broadway. When you get to the intersection of Broadway and Elm, you hang a left. Go straight down that st...",<pad> #Person1# shows #Person2# how to get to the Cross Bakery. #Person2# tells #Person1# the direction #Person1# should take. After making a left you'll see the Cross Bakery on the left side.</s>,<pad> #Person1# tells #Person2# how to get to the Cross Bakery. #Person2# tells #Person1# the way to the cross bakery and tells #Person1# how to get there.</s>,2.969739,3.206602,0.236863
4,"Summarize the following conversation. #Person1#: Let's take a coffee break, shall we? #Person2#: I wish I could, but I can't. #Person1#: What keeps you so busy? You've been sitting there for hours. You've got to walk around. You just can't stay on the computer forever. #Person2#: Well, I am up to my neck in work. I've got to finish this report. Sarah needs it by noon. I don't want to be scolded if I can't finish my work by the deadline. #Person1#: I un

Olhando para a média/mediana de recompensa das sequências geradas você pode observar uma diferença significativa!